In [ ]:
# --- FIX LỖI MÔI TRƯỜNG KAGGLE ---
# Lệnh này sẽ chạy trên server Kaggle để gỡ thư viện gây xung đột
import os

# Gỡ bỏ Cupy (thủ phạm chính gây xung đột với Spacy/Scipy hiện tại)
os.system("pip uninstall -y cupy cupy-cuda11x cupy-cuda12x")

# Cài đặt lại Numpy và Scipy bản ổn định để tránh lỗi "ufunc"
os.system("pip install 'numpy<2.0' 'scipy<1.13'")

# --- KẾT THÚC FIX LỖI ---
print("Đã sửa xong môi trường! Bắt đầu chạy code chính...")

In [ ]:
# Thông tin thiết bị (GPU/CPU) và phiên bản Torch
import torch, platform
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Thiết bị đang dùng: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Số GPU khả dụng: {torch.cuda.device_count()}")
    print(f"Tên GPU[0]: {torch.cuda.get_device_name(0)}")
print(f"Torch version: {torch.__version__}")
print(f"Python: {platform.python_version()} | Platform: {platform.platform()}")

In [ ]:
import os
import requests
import gzip
import shutil

# 1. Cấu hình thư mục
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Đã tạo thư mục: {data_dir}")

# 2. Danh sách link file NÉN (.gz) chính thức hiện tại
# Lưu ý: Các file này đều có đuôi .gz
urls = {
    "train.en": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.en.gz",
    "train.fr": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.fr.gz",
    "val.en": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.en.gz",
    "val.fr": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.fr.gz",
    "test.en": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.en.gz",
    "test.fr": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.fr.gz"
}

print("Đang bắt đầu tải và giải nén dữ liệu...")

for filename, url in urls.items():
    # Đường dẫn file nén (.gz) và file đích (txt)
    gz_path = os.path.join(data_dir, filename + ".gz") # ví dụ: train.en.gz
    final_path = os.path.join(data_dir, filename)      # ví dụ: train.en
    
    # Kiểm tra nếu file đích chưa có thì mới tải
    if not os.path.exists(final_path):
        print(f"--> Đang tải: {filename}...")
        
        try:
            # 1. Tải file .gz về
            response = requests.get(url)
            if response.status_code == 200:
                with open(gz_path, 'wb') as f:
                    f.write(response.content)
                
                # 2. Giải nén file .gz thành file text
                with gzip.open(gz_path, 'rb') as f_in:
                    with open(final_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                
                # 3. Xóa file .gz cho nhẹ máy
                os.remove(gz_path)
                print(f"    Đã tải và giải nén xong: {filename}")
            else:
                print(f"    LỖI: Link hỏng hoặc file không tồn tại (Status {response.status_code})")
                
        except Exception as e:
            print(f"    Lỗi ngoại lệ khi xử lý {filename}: {e}")
    else:
        print(f"--> File {filename} đã tồn tại, bỏ qua.")

# 3. Kiểm tra kết quả
print("\nDanh sách file trong thư mục data/ (Kiểm tra xem dung lượng có > 0KB không):")
files = os.listdir(data_dir)
for f in files:
    size = os.path.getsize(os.path.join(data_dir, f))
    print(f"- {f}: {size/1024:.2f} KB")

# Test thử nội dung file train.en xem có phải tiếng Anh không
print("\n--- Kiểm tra nội dung file train.en (5 dòng đầu) ---")
try:
    with open(os.path.join(data_dir, 'train.en'), 'r', encoding='utf-8') as f:
        for i in range(5):
            print(f.readline().strip())
except Exception as e:
    print(f"Chưa đọc được file: {e}")

In [ ]:
# Khởi động môi trường PyTorch an toàn (tắt Dynamo/ONNX trước khi import torch)
import os
os.environ["PYTORCH_ENABLE_DYNAMO"] = "0"
os.environ["TORCH_ONNX_DISABLE"] = "1"
print("Đã set PYTORCH_ENABLE_DYNAMO=0 và TORCH_ONNX_DISABLE=1. Hãy Restart Kernel rồi chạy lại notebook từ đầu.")


In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import spacy
import io

# --- Simple Vocabulary (no torchtext) ---
class Vocabulary:
    def __init__(self, counter, min_freq=2, specials=('\u003Cunk\u003E','\u003Cpad\u003E','\u003Csos\u003E','\u003Ceos\u003E')):
        self.specials = list(specials)
        self.itos = self.specials.copy()
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}
        for token, freq in counter.items():
            if freq >= min_freq and token not in self.stoi:
                self.stoi[token] = len(self.itos)
                self.itos.append(token)
        self.default_index = self.stoi['\u003Cunk\u003E']
    def __getitem__(self, token):
        return self.stoi.get(token, self.default_index)
    def __len__(self):
        return len(self.itos)

# --- 1. TOKENIZER (SPACY) ---
print("Đang tải Spacy models...")
spacy_en = spacy.load("en_core_web_sm")
spacy_fr = spacy.load("fr_core_news_sm")

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

# --- 2. BUILD VOCAB ---
def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            counter.update(tokenizer(string_.lower().strip()))
    return Vocabulary(counter, min_freq=2)

print("Đang xây dựng từ điển (Vocabulary)...")
en_vocab = build_vocab('data/train.en', tokenize_en)
fr_vocab = build_vocab('data/train.fr', tokenize_fr)

print(f"- Số lượng từ vựng Tiếng Anh: {len(en_vocab)}")
print(f"- Số lượng từ vựng Tiếng Pháp: {len(fr_vocab)}")

PAD_IDX = en_vocab['\u003Cpad\u003E']
SOS_IDX = en_vocab['\u003Csos\u003E']
EOS_IDX = en_vocab['\u003Ceos\u003E']

# --- 3. DATASET ---
class Multi30kDataset(Dataset):
    def __init__(self, src_path, trg_path, src_vocab, trg_vocab, src_tokenizer, trg_tokenizer):
        self.src_data = open(src_path, encoding='utf-8').read().split('\n')
        self.trg_data = open(trg_path, encoding='utf-8').read().split('\n')
        min_len = min(len(self.src_data), len(self.trg_data))
        self.src_data = self.src_data[:min_len]
        self.trg_data = self.trg_data[:min_len]
        self.src_data, self.trg_data = zip(*[(s, t) for s, t in zip(self.src_data, self.trg_data) if s.strip() and t.strip()])
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_text = self.src_data[idx].lower().strip()
        trg_text = self.trg_data[idx].lower().strip()
        src_indices = [self.src_vocab[token] for token in self.src_tokenizer(src_text)]
        trg_indices = [self.trg_vocab[token] for token in self.trg_tokenizer(trg_text)]
        src_indices = [SOS_IDX] + src_indices + [EOS_IDX]
        trg_indices = [SOS_IDX] + trg_indices + [EOS_IDX]
        return torch.tensor(src_indices), torch.tensor(trg_indices)

# --- 4. COLLATE ---
def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src_item, trg_item in batch:
        src_batch.append(src_item)
        trg_batch.append(trg_item)
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=False)
    trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX, batch_first=False)
    return src_batch, trg_batch

# --- 5. DATALOADER ---
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Đang tạo DataLoaders (Batch Size: {BATCH_SIZE})...")
train_dataset = Multi30kDataset('data/train.en', 'data/train.fr', en_vocab, fr_vocab, tokenize_en, tokenize_fr)
val_dataset = Multi30kDataset('data/val.en', 'data/val.fr', en_vocab, fr_vocab, tokenize_en, tokenize_fr)
test_dataset = Multi30kDataset('data/test.en', 'data/test.fr', en_vocab, fr_vocab, tokenize_en, tokenize_fr)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# --- 6. SAMPLE BATCH ---
src, trg = next(iter(train_loader))
print("\n--- Kiểm tra cấu trúc Batch ---")
print(f"Shape Source (seq_len, batch_size): {src.shape}")
print(f"Shape Target (seq_len, batch_size): {trg.shape}")
print("Done! Dữ liệu đã sẵn sàng để train.")

# Xây dựng mô hình LSTM Seq2Seq

Phần này triển khai Encoder–Decoder LSTM với các tham số có thể chỉnh sửa.

- Bạn có thể sửa các tham số trong mục "Config" ở đầu cell tiếp theo (hidden size, embedding dim, số layer, dropout, tỉ lệ teacher forcing, batch size, số epoch, learning rate, v.v.).
- Loop huấn luyện in ra train/val loss theo epoch và lưu `best_model.pt` khi val loss giảm.
- Hàm `translate(sentence)` ở cuối để suy luận nhanh trên câu tiếng Anh.

In [ ]:
# Config: chỉnh các tham số tuỳ ý
CONFIG = {
    "vocab_en": None,   # sẽ set từ cell preprocessing
    "vocab_fr": None,   # sẽ set từ cell preprocessing
    "pad_idx": None,    # sẽ set từ cell preprocessing
    "sos_idx": None,    # sẽ set từ cell preprocessing
    "eos_idx": None,    # sẽ set từ cell preprocessing
    "embedding_dim": 512,   # 256–512
    "hidden_size": 1024,     
    "num_layers": 8,
    "dropout": 0.3,         # 0.3–0.5
    "teacher_forcing": 0.5,
    "lr": 1e-35,
    "epochs": 10,           # 10–20
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

# Lấy các đối tượng/từ cell preprocess
CONFIG["vocab_en"] = en_vocab
CONFIG["vocab_fr"] = fr_vocab
CONFIG["pad_idx"] = PAD_IDX
CONFIG["sos_idx"] = SOS_IDX
CONFIG["eos_idx"] = EOS_IDX

import torch.nn as nn
import random

class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, num_layers, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(emb_dim, hidden_size, num_layers=num_layers, dropout=dropout, bidirectional=False)
    def forward(self, src):  # src: [seq_len, batch]
        embedded = self.embedding(src)  # [seq_len, batch, emb_dim]
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs: [seq_len, batch, hidden]; hidden/cell: [num_layers, batch, hidden]
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, num_layers, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(emb_dim, hidden_size, num_layers=num_layers, dropout=dropout, bidirectional=False)
        self.fc = nn.Linear(hidden_size, vocab_size)
    def forward(self, input_token, hidden, cell):  # input_token: [batch]
        input_token = input_token.unsqueeze(0)     # [1, batch]
        embedded = self.embedding(input_token)     # [1, batch, emb_dim]
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        logits = self.fc(output.squeeze(0))        # [batch, vocab_size]
        return logits, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, sos_idx, eos_idx, pad_idx, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        self.device = device
    def forward(self, src, trg, teacher_forcing=0.5):
        # src: [src_len, batch], trg: [trg_len, batch]
        batch = src.size(1)
        trg_len = trg.size(0)
        vocab_size = self.decoder.fc.out_features
        outputs = torch.zeros(trg_len, batch, vocab_size, device=self.device)
        hidden, cell = self.encoder(src)
        input_token = torch.full((batch,), self.sos_idx, dtype=torch.long, device=self.device)
        for t in range(trg_len):
            logits, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[t] = logits
            teacher = random.random() < teacher_forcing
            input_token = trg[t] if teacher else torch.argmax(logits, dim=1)
        return outputs

# Khởi tạo model
enc = Encoder(
    vocab_size=len(CONFIG["vocab_en"]),
    emb_dim=CONFIG["embedding_dim"],
    hidden_size=CONFIG["hidden_size"],
    num_layers=CONFIG["num_layers"],
    dropout=CONFIG["dropout"],
    pad_idx=CONFIG["pad_idx"],
)

dec = Decoder(
    vocab_size=len(CONFIG["vocab_fr"]),
    emb_dim=CONFIG["embedding_dim"],
    hidden_size=CONFIG["hidden_size"],
    num_layers=CONFIG["num_layers"],
    dropout=CONFIG["dropout"],
    pad_idx=CONFIG["pad_idx"],
)

model = Seq2Seq(enc, dec, CONFIG["sos_idx"], CONFIG["eos_idx"], CONFIG["pad_idx"], CONFIG["device"]).to(CONFIG["device"])

In [ ]:
# Khắc phục lỗi Torch Dynamo/ONNX import trên Anaconda
import os
os.environ["PYTORCH_ENABLE_DYNAMO"] = "0"  # Tắt TorchDynamo trước khi import torch
# os.environ["TORCH_ONNX_DISABLE"] = "1"  # Tuỳ chọn: tắt các phần ONNX nếu cần

import torch
print("Đã tắt TorchDynamo qua biến môi trường. Tiếp tục cấu hình mô hình...")


In [ ]:
# Nếu dùng CUDA 12.x
!pip install --upgrade cupy-cuda12x

# Hoặc nếu dùng CUDA 11.x
!pip install --upgrade cupy-cuda11x

In [ ]:
# Huấn luyện (có Early Stopping và Scheduler ReduceLROnPlateau)
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss(ignore_index=CONFIG["pad_idx"])  # bỏ qua pad
optimizer = optim.Adam(model.parameters(), lr=CONFIG["lr"])

# Scheduler: giảm learning rate khi val_loss không cải thiện
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, min_lr=1e-6, verbose=True)

best_val = float('inf')
epochs_no_improve = 0  # Early Stopping: đếm số epoch không cải thiện
early_stopping_patience = 3  # Dừng sớm nếu không giảm sau 3 epoch (có thể chỉnh)

def run_epoch(dataloader, train=True):
    total_loss = 0.0
    model.train(train)
    with torch.set_grad_enabled(train):
        for src_batch, trg_batch in dataloader:
            src_batch = src_batch.to(CONFIG["device"])  # [src_len, batch]
            trg_batch = trg_batch.to(CONFIG["device"])  # [trg_len, batch]
            outputs = model(src_batch, trg_batch, teacher_forcing=CONFIG["teacher_forcing"])  # [trg_len, batch, vocab]
            # Dịch mục tiêu sang next-token (bỏ token đầu)
            logits = outputs[:-1].reshape(-1, outputs.size(-1))
            target = trg_batch[1:].reshape(-1)
            loss = criterion(logits, target)
            if train:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
            total_loss += loss.item()
    return total_loss / max(1, len(dataloader))

for epoch in range(1, CONFIG["epochs"] + 1):
    train_loss = run_epoch(train_loader, train=True)
    val_loss = run_epoch(val_loader, train=False)

    # Bước scheduler dựa trên val_loss
    scheduler.step(val_loss)

    print(f"Epoch {epoch}/{CONFIG['epochs']} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | lr={optimizer.param_groups[0]['lr']:.2e}")

    # Early Stopping & lưu best model
    if val_loss < best_val:
        best_val = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pt")
        print("→ Saved best_model.pt (val_loss improved)")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epoch(s)")
        if epochs_no_improve >= early_stopping_patience:
            print(f"Early stopping triggered (patience={early_stopping_patience}).")
            break

In [ ]:
# Suy luận (Inference)

def detokenize_fr(indices, vocab_fr):
    # Bỏ các specials và dừng ở <eos>
    itos = vocab_fr.itos
    tokens = []
    for idx in indices:
        tok = itos[idx]
        if tok in ("\u003Cpad\u003E", "\u003Csos\u003E"):
            continue
        if tok == "\u003Ceos\u003E":
            break
        tokens.append(tok)
    return " ".join(tokens)

@torch.no_grad()
def translate(sentence_en: str, max_len: int = 50):
    model.eval()
    device = CONFIG["device"]
    src_tokens = tokenize_en(sentence_en.lower().strip())
    src_indices = [CONFIG["sos_idx"]] + [CONFIG["vocab_en"][t] for t in src_tokens] + [CONFIG["eos_idx"]]
    src = torch.tensor(src_indices, dtype=torch.long, device=device).unsqueeze(1)  # [seq_len, 1]
    hidden, cell = model.encoder(src)
    cur = torch.tensor([CONFIG["sos_idx"]], dtype=torch.long, device=device)  # [1]
    out_indices = []
    for _ in range(max_len):
        logits, hidden, cell = model.decoder(cur, hidden, cell)
        cur = torch.argmax(logits, dim=1)
        out_indices.append(cur.item())
        if cur.item() == CONFIG["eos_idx"]:
            break
    return detokenize_fr(out_indices, CONFIG["vocab_fr"])

In [ ]:
# # Đánh giá (Evaluation): BLEU & Perplexity
# import math
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# # Tính BLEU trung bình trên tập test
# def compute_bleu_on_test(max_samples: int = None):
#     model.eval()
#     smoothie = SmoothingFunction().method3
#     total_bleu = 0.0
#     count = 0
#     examples = []  # lưu 5 ví dụ minh hoạ
#     # Duyệt dữ liệu thô để lấy cặp câu gốc
#     src_lines = open('data/test.en', encoding='utf-8').read().strip().split('\n')
#     trg_lines = open('data/test.fr', encoding='utf-8').read().strip().split('\n')
#     n = min(len(src_lines), len(trg_lines))
#     if max_samples is not None:
#         n = min(n, max_samples)
#     for i in range(n):
#         src = src_lines[i].strip()
#         trg = trg_lines[i].strip()
#         if not src or not trg:
#             continue
#         # Dịch bằng model (greedy)
#         pred = translate(src, max_len=50)
#         # Token hoá tham chiếu và dự đoán (sử dụng tokenizer spaCy đã có)
#         ref_tokens = tokenize_fr(trg.lower().strip())
#         hyp_tokens = pred.lower().strip().split()
#         # BLEU câu
#         bleu_i = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothie)
#         total_bleu += bleu_i
#         count += 1
#         if len(examples) < 5:
#             examples.append({
#                 'src': src,
#                 'ref': trg,
#                 'hyp': pred,
#                 'bleu': bleu_i,
#             })
#     avg_bleu = total_bleu / max(1, count)
#     return avg_bleu, examples

# # Perplexity từ loss (ví dụ dùng val_loss cuối cùng)
# def loss_to_perplexity(loss_value: float):
#     try:
#         return math.exp(loss_value)
#     except OverflowError:
#         return float('inf')

# # Chạy đánh giá và in kết quả
# avg_bleu, samples = compute_bleu_on_test(max_samples=1000)  # có thể giảm số lượng để nhanh hơn
# print(f"BLEU trung bình (test, max_samples=1000): {avg_bleu:.4f}")

# try:
#     # nếu biến best_val tồn tại từ cell train, tính perplexity
#     ppl = loss_to_perplexity(best_val)
#     print(f"Perplexity (ước từ best val_loss): {ppl:.2f}")
# except NameError:
#     print("Perplexity: chưa có 'best_val' (hãy chạy cell huấn luyện)")

# print("\n— 5 ví dụ minh hoạ —")
# for i, ex in enumerate(samples, 1):
#     print(f"[{i}] EN: {ex['src']}")
#     print(f"    REF_FR: {ex['ref']}")
#     print(f"    HYP_FR: {ex['hyp']}")
#     print(f"    BLEU: {ex['bleu']:.4f}\n")

In [ ]:
# Tạo alias cho bộ test 2016 nếu thiếu (dùng test.en/test.fr)
import os, shutil
aliases = [
    ('data/test.en', 'data/test_2016_flickr.en'),
    ('data/test.fr', 'data/test_2016_flickr.fr'),
]
for src, dst in aliases:
    if not os.path.exists(dst):
        if os.path.exists(src):
            shutil.copyfile(src, dst)
            print(f"Đã tạo: {dst} từ {src}")
        else:
            print(f"Thiếu nguồn: {src} — hãy chạy cell tải dữ liệu đầu tiên")
    else:
        print(f"Đã có: {dst}")

In [ ]:
# Tải thêm các bộ test năm 2017 và 2018 (nếu có)
import os, requests, gzip, shutil
data_dir = 'data'
extra_urls = {
    "test_2017_flickr.en": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2017_flickr.en.gz",
    "test_2017_flickr.fr": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2017_flickr.fr.gz",
    "test_2018_flickr.en": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2018_flickr.en.gz",
    "test_2018_flickr.fr": "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2018_flickr.fr.gz",
}
print("Đang kiểm tra và tải thêm bộ test (2017/2018) nếu cần...")
for filename, url in extra_urls.items():
    gz_path = os.path.join(data_dir, filename + ".gz")
    final_path = os.path.join(data_dir, filename)
    if os.path.exists(final_path):
        print(f"- Đã có {filename}, bỏ qua.")
        continue
    try:
        r = requests.get(url)
        if r.status_code == 200:
            with open(gz_path, 'wb') as f: f.write(r.content)
            with gzip.open(gz_path, 'rb') as f_in, open(final_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
            os.remove(gz_path)
            print(f"- Tải xong: {filename}")
        else:
            print(f"- Không tải được {filename} (HTTP {r.status_code})")
    except Exception as e:
        print(f"- Lỗi khi tải {filename}: {e}")

In [ ]:
# Đánh giá đa bộ test: 2016 / 2017 / 2018
import math
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from pathlib import Path
@torch.no_grad()
def compute_bleu_for_files(src_path: str, trg_path: str, max_samples: int = None):
    model.eval()
    smoothie = SmoothingFunction().method3
    total_bleu, count = 0.0, 0
    examples = []
    if not (Path(src_path).exists() and Path(trg_path).exists()):
        print(f"Không tìm thấy file: {src_path} hoặc {trg_path}")
        return None, []
    src_lines = open(src_path, encoding='utf-8').read().strip().split('\n')
    trg_lines = open(trg_path, encoding='utf-8').read().strip().split('\n')
    n = min(len(src_lines), len(trg_lines))
    if max_samples is not None:
        n = min(n, max_samples)
    for i in range(n):
        src = src_lines[i].strip()
        ref = trg_lines[i].strip()
        if not src or not ref:
            continue
        hyp = translate(src, max_len=50)
        try:
            ref_tokens = tokenize_fr(ref.lower().strip())  # dùng spaCy nếu có
        except Exception:
            ref_tokens = ref.lower().strip().split()       # fallback đơn giản
        hyp_tokens = hyp.lower().strip().split()
        bleu_i = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothie)
        total_bleu += bleu_i
        count += 1
        if len(examples) < 5:
            examples.append({'src': src, 'ref': ref, 'hyp': hyp, 'bleu': bleu_i})
    if count == 0:
        return 0.0, examples
    return total_bleu / count, examples
def loss_to_perplexity(loss_value: float):
    try:
        return math.exp(loss_value)
    except OverflowError:
        return float('inf')
# Cố gắng load best_model nếu có
try:
    model.load_state_dict(torch.load('best_model.pt', map_location=CONFIG['device']))
    model.eval()
    print('Đã load: best_model.pt')
except Exception as e:
    print(f"Không load được best_model.pt (tiếp tục dùng model hiện tại): {e}")
# Danh sách bộ test
test_sets = {
    '2016_flickr': ('data/test_2016_flickr.en', 'data/test_2016_flickr.fr'),
    '2017_flickr': ('data/test_2017_flickr.en', 'data/test_2017_flickr.fr'),
    '2018_flickr': ('data/test_2018_flickr.en', 'data/test_2018_flickr.fr'),
}
results = {}
for name, (src_p, trg_p) in test_sets.items():
    print(f"\n=== Đánh giá {name} ===")
    avg_bleu, examples = compute_bleu_for_files(src_p, trg_p, max_samples=1000)
    if avg_bleu is None:
        continue
    results[name] = avg_bleu
    print(f"BLEU trung bình: {avg_bleu:.4f}")
    try:
        ppl = loss_to_perplexity(best_val)
        print(f"Perplexity (ước từ best val_loss): {ppl:.2f}")
    except NameError:
        print("Perplexity: chưa có 'best_val' (hãy chạy cell huấn luyện)")
    print("— 3 ví dụ minh hoạ —")
    for i, ex in enumerate(examples[:3], 1):
        print(f"[{i}] EN: {ex['src']}")
        print(f"    REF_FR: {ex['ref']}")
        print(f"    HYP_FR: {ex['hyp']}")
        print(f"    BLEU: {ex['bleu']:.4f}\n")
print("\nTổng hợp BLEU theo bộ test:")
for k, v in results.items():
    print(f"- {k}: {v:.4f}")